In [29]:
import os 
import sys

sys.path.append("..")

import numpy as np
import keras.backend as K
from train import SqueezeNet

from preparator import load_reduce_pipeline

import matplotlib.pyplot as plt
%matplotlib inline

import scipy

In [21]:
with open("data/train/videos/ideal.txt") as f:
    xy = {}
    
    for line in f.readlines():
        x_y = line.split("  ")
        xy[x_y[0]] = int(x_y[1])

In [ ]:
model = SqueezeNet(3, (224, 224, 3))
model.load_weights("trained_model/challenge1.weights")

In [18]:
vid_name = "data/public_test/videos/"+np.random.choice(os.listdir("data/train/videos/"), 1)[0]
vid = load_reduce_pipeline(vid_name)

OSError: [Errno 2] No such file or directory: 'data/public_test//videos/akn.151.157.left.avi'

In [5]:
def predict_for_video(vid):
    predictions = list()
    
    for frame in vid:
        predictions.append(model.predict(np.array([frame])).argmax(axis=1)[0])
        
    return predictions

predictions = predict_for_video(vid)

In [90]:
def blurring(x, k_size=7):
    x = np.array(x)
    x_new = np.zeros(x.shape, "int32")
    for i in range(len(x)):
        x_new[i] =  scipy.stats.mode(x[i : i + k_size])[0][0]
        
    return x_new

In [95]:
def custom_kernel_switch(predictions):
    predictions = np.array(predictions)
    predictions = blurring(predictions)
    kernel = np.array([-1, -1, -1, 1, 1, 1]) # 3 -1 : 3 1
    scores = list()

    for i in range(len(predictions) - len(kernel)):
        scores.append(np.sum(predictions[i : i + len(kernel)] * kernel))
        
    maximum = np.array(scores).argmax()
    if  maximum > 0:
        return maximum
    
    return -1

In [25]:
def predict_validate(path, num_samps=5, finder_pipe=find_switch_frame):
    predictions = list()
    
    vid_names = np.random.choice(filter(lambda x: x.split(".")[-1] == "avi", os.listdir(path)), 
                                 size=num_samps, replace=False)
    for vid_name in vid_names:
        vid = load_reduce_pipeline(path+vid_name)
        preds = predict_for_video(vid)
        
        predictions.append(preds)
        
        print vid_name
        print finder_pipe(preds), xy[vid_name]
    
    return predictions

In [96]:
preds = predict_validate("data/train/videos/", num_samps=15, finder_pipe=custom_kernel_switch)

akn.025.026.left.avi
211 218
akn.024.019.left.avi
61 -1
akn.033.084.left.avi
11 -1
akn.155.084.left.avi
-1 -1
akn.154.021.left.avi
195 204
akn.019.002.left.avi
260 -1
akn.151.157.left.avi
1 214
akn.060.018.left.avi
85 90
akn.025.050.left.avi
-1 -1
akn.154.216.left.avi
132 -1
akn.019.012.left.avi
30 -1
akn.025.027.left.avi
58 -1
akn.021.024.left.avi
105 -1
akn.020.036.left.avi
42 -1
akn.151.076.left.avi
-1 -1


In [112]:
def predict_validate(path, finder_pipe=find_switch_frame):
    predictions = list()
    a=0
    
    vid_names = filter(lambda x: x.split(".")[-1] == "avi", os.listdir(path))
    
    for vid_name in vid_names:
        a+=1
        print a
        vid = load_reduce_pipeline(path+vid_name)
        preds = predict_for_video(vid)
        
        predictions.append(finder_pipe(preds))
        
        
    return predictions